In [12]:
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = 'Jurassic Park (novel)'

PARAMS = {
    'action': "query",
    'titles': TITLE,
    'prop': 'links',
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
data = R.json()
#print(data['parse']['title'])

In [14]:
data['query']

{'pages': {'5326809': {'pageid': 5326809,
   'ns': 0,
   'title': 'Jurassic Park (novel)',
   'links': [{'ns': 0, 'title': '2000 AD (comics)'},
    {'ns': 0, 'title': 'A Case of Need'},
    {'ns': 0, 'title': 'Airframe (novel)'},
    {'ns': 0, 'title': 'Alfred A. Knopf'},
    {'ns': 0, 'title': 'Amazon (video game)'},
    {'ns': 0, 'title': 'Amber'},
    {'ns': 0, 'title': 'Amphibian'},
    {'ns': 0, 'title': 'Amusement park'},
    {'ns': 0, 'title': 'Ancient DNA'},
    {'ns': 0, 'title': 'Andrew Ferguson'}]}}}